# Multi Criteria Software Component Asssigment Problem Solver

The core challenge lies in minimizing conflicting objectives (e.g., communication cost vs. hardware cost) while ensuring strict safety and resource constraints are met.

## Dummy Data Generation

### Hardware Features:
1) **HW_HSM (Hardware Security Module)**: a dedicated physical device that securely stores, manages, and uses cryptographic keys
2) **HW_FLEX (FlexRay Controller)**: a hardware component that executes the FlexRay communication protocol, handling all tasks related to data transmission
3) **HW_CANFD (CAN Flexible Data-rate)** : Classic CAN
4) **HW_ETH (Automotive Ethernet)** : Classic Ethernet

### ECU Details:
We have ECU tiers: **HPC**, **Gateway**, **Safety** those are not directly taking care of by the optimization but those are kinda template to determine features of the ECUs.

### Component Types:
We have component types, thoose are also not taking into account again those are just template 
1) **PERCEPTION**: AI etc, High computation power, High Bandwidth, Requires a **GPU**, **HW_ETH** would require, ASIL b/w **QM** and **B**.
2) **CONTROL**: Chassis, Brake, Motor, Hard Real-Time (Deterministic), **HW_FLEX** / **HW_HSM**, low CPU requirement, ASIL D.
3) **BODY**: Klima, Doors, Lights., Low Resources, ASIL QM/A/B. 





1) **CPU**: Unit is Instructions Per Second
2) **GPU**: Unit is FLOPS
3) **RAM**: Unit is MiB
4) **ROM**: Unit is MiB


* Also we need to min req for CPU and GPU etc.


In [57]:
from gekko import GEKKO
import numpy as np
import random
from typing import List, Dict, Tuple

# --- Configuration for Generation ---
NUM_COMPONENTS = 40   
NUM_ECUS = 10          
RANDOM_SEED = 42 

# Definitions 
COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]
M = len(COMPONENTS)

# --- Percentages for Generation ---
# Supply (ECUs) - Counts
COUNT_HPC = 3
COUNT_SAFETY = 4
COUNT_GATEWAY = 3

# Demand (Components) - Weights
WEIGHT_PERCEPTION = 0.25
WEIGHT_CONTROL = 0.25
WEIGHT_BODY = 0.30
WEIGHT_INFOTAINMENT = 0.2  

# ASIL Mapping 
ASIL_MAP = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}

def generate_robust_data(cnt_hpc, cnt_safety, cnt_gateway, w_perception, w_control, w_body, w_infotainment):
    """Generates synthetic data"""
    
    if RANDOM_SEED is not None:
        random.seed(RANDOM_SEED)
    
    # --- 1. Define ECU Profiles (The Supply) ---
    # Create list based on counts
    ecu_types_list = ['HPC']*cnt_hpc + ['SAFETY']*cnt_safety + ['GATEWAY']*cnt_gateway
    
    # Fill remainder if NUM_ECUS > sum of counts
    while len(ecu_types_list) < NUM_ECUS:
        ecu_types_list.append(random.choice(['HPC', 'SAFETY', 'GATEWAY']))
    
    random.shuffle(ecu_types_list) 
    
    # Initialize Dictionaries with High-Fidelity Units
    max_cpu, max_ram, max_rom, max_gpu = {}, {}, {}, {}
    base_cost = {}
    ecu_asil = {}
    ecu_caps = {}
    ecu_tiers = {}
    
    for idx, tier in enumerate(ecu_types_list):
        j = ECUS[idx] 
        ecu_tiers[j] = tier
        
        if tier == 'HPC':
            # REALISTIC SPECS: Orin/Snapdragon Class
            max_cpu[j] = 250000.0  # 250k DMIPS
            max_gpu[j] = 500.0     # 500 TOPS (AI)
            max_ram[j] = 65536.0   # 64 GB RAM
            max_rom[j] = 256000.0  # 256 GB Storage
            base_cost[j] = 200.0
            ecu_asil[j] = 'B' 
            ecu_caps[j] = ['HW_ETH']
            
        elif tier == 'SAFETY':
            # REALISTIC SPECS: Aurix TC4x Class
            max_cpu[j] = 16000.0   # 16k DMIPS
            max_gpu[j] = 0.0       # No AI Accel
            max_ram[j] = 24.0      # 24 MB SRAM
            max_rom[j] = 64.0      # 64 MB Flash
            base_cost[j] = 80.0
            ecu_asil[j] = 'D' 
            ecu_caps[j] = ['HW_HSM', 'HW_FLEX', 'HW_CANFD']
            
        else: # GATEWAY
            # REALISTIC SPECS: S32G Class
            max_cpu[j] = 8000.0    # 8k DMIPS
            max_gpu[j] = 0.0
            max_ram[j] = 8.0       # 8 MB SRAM
            max_rom[j] = 16.0      # 16 MB Flash
            base_cost[j] = 30.0
            ecu_asil[j] = 'B'
            ecu_caps[j] = ['HW_HSM', 'HW_CANFD','HW_ETH']

    # --- 2. Define Component Profiles (The Demand) ---
    req_cpu, req_ram, req_rom, req_gpu = {}, {}, {}, {}
    req_asil, req_hw = {}, {}
    comp_types = {} 
    
    for i in COMPONENTS:
        # Use the weights passed as arguments
        ctype = random.choices(
            ['PERCEPTION', 'CONTROL', 'BODY', "INFOTAINMENT"], 
            weights=[w_perception, w_control, w_body, w_infotainment] 
        )[0]
        comp_types[i] = ctype
        
        if ctype == 'PERCEPTION':
            # High Load AI Task
            req_cpu[i] = random.uniform(15000, 40000) # 15k-40k DMIPS
            req_gpu[i] = random.uniform(20, 150)       # 20-80 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)   # 2-8 GB RAM
            req_rom[i] = random.uniform(1000, 5000)   # 1-5 GB Storage
            req_asil[i] = random.choice(['QM', 'B'])
            req_hw[i] = ['HW_ETH']
            
        elif ctype == 'CONTROL':
            # Critical Real-Time Task
            req_cpu[i] = random.uniform(500, 3000)    # 500-3k DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(1, 5)         # 1-5 MB
            req_rom[i] = random.uniform(2, 10)        # 2-10 MB
            req_asil[i] = 'D' 
            req_hw[i] = ['HW_FLEX'] 

        elif ctype == 'INFOTAINMENT':
            # Medium Load Task
            req_cpu[i] = random.uniform(5000, 20000)  # 2k-10k DMIPS
            req_gpu[i] = random.uniform(10, 80)        # 1-10 TOPS
            req_ram[i] = random.uniform(2048*2, 8192*2)    # 512 MB - 2 GB
            req_rom[i] = random.uniform(1000, 2000)    # 500 MB - 2 GB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = ['HW_ETH']
            
        else: # BODY
            # Simple Logic
            req_cpu[i] = random.uniform(50, 300)      # 50-300 DMIPS
            req_gpu[i] = 0.0
            req_ram[i] = random.uniform(0.1, 1.0)     # <1 MB
            req_rom[i] = random.uniform(0.5, 2.0)     # <2 MB
            req_asil[i] = random.choice(['QM', 'A'])
            req_hw[i] = [] 

    # --- 3. Comm Matrix ---
    comm_vol = {}
    for i in COMPONENTS:
        for k in COMPONENTS:
            if i < k and random.random() < 0.1: # 10% chance of communication 
                comm_vol[(i, k)] = random.randint(5, 50)
                comm_vol[(k, i)] = comm_vol[(i, k)]

    # --- 4. Critical Pairs ---
    critical_pairs = []
    safety_comps = [c for c, a in req_asil.items() if a == 'D']
    if len(safety_comps) >= 2:
        c1, c2 = random.sample(safety_comps, 2)
        critical_pairs.append((c1, c2))

    return (max_cpu, max_ram, max_rom, max_gpu, base_cost, ecu_caps, ecu_asil, ecu_tiers,
            req_cpu, req_ram, req_rom, req_gpu, req_asil, req_hw, comp_types,
            comm_vol, critical_pairs)

In [58]:
# --- EXECUTE GENERATION ---
# 1. Call the function with the correct parameters
# 2. Unpack ALL 17 return values including the new ROM/GPU data
(MAX_CPU, MAX_RAM, MAX_ROM, MAX_GPU, ECU_BASE_COST, ECU_HW_CAPABILITIES, ECU_ASIL_CAPABILITY, ECU_TIERS,
 REQ_CPU, REQ_RAM, REQ_ROM, REQ_GPU, REQ_ASIL, REQ_HW, COMP_TYPES,
 COMM_VOLUME, CRITICAL_PAIRS_TO_SEPARATE) = generate_robust_data(
    COUNT_HPC, COUNT_SAFETY, COUNT_GATEWAY, 
    WEIGHT_PERCEPTION, WEIGHT_CONTROL, WEIGHT_BODY, WEIGHT_INFOTAINMENT
)

# --- PRE-SOLVE DIAGNOSTICS ---
print(f"\n{'='*60}")
print(f"      SCENARIO SUMMARY (Seed: {RANDOM_SEED})")
print(f"{'='*60}")

# Check 1: CPU Supply vs Demand (DMIPS)
total_cpu_supply = sum(MAX_CPU.values())
total_cpu_demand = sum(REQ_CPU.values())
print(f"CPU Supply (DMIPS): {total_cpu_supply:,.0f} | Demand: {total_cpu_demand:,.0f}")
if total_cpu_demand > total_cpu_supply:
    print("FAILURE: Global CPU Shortage")

# Check 2: GPU Supply vs Demand (TOPS)
# Now we compare actual TOPS, not just CPU power of GPU nodes.
total_gpu_supply = sum(MAX_GPU.values())
total_gpu_demand = sum(REQ_GPU.values())
print(f"GPU Supply (TOPS):  {total_gpu_supply:,.0f} | Demand: {total_gpu_demand:,.0f}")

if total_gpu_demand > total_gpu_supply:
    print("FAILURE: Not enough AI Acceleration (TOPS)!")
elif total_gpu_demand > total_gpu_supply * 0.9:
    print("WARNING: GPU Saturation > 90%")
else:
    print("GPU Supply Healthy")

# Check 3: ASIL D Constraints
safety_ecus = [e for e, lvl in ECU_ASIL_CAPABILITY.items() if lvl == 'D']
safety_comps = [c for c, lvl in REQ_ASIL.items() if lvl == 'D']
saf_supply = sum(MAX_CPU[e] for e in safety_ecus)
saf_demand = sum(REQ_CPU[c] for c in safety_comps)

print(f"Safety Nodes: {len(safety_ecus)} ECUs | Critical Tasks: {len(safety_comps)}")
if saf_demand > saf_supply:
     print("FAILURE: Safety Tier CPU Overload")
else:
     print("Safety Tier Supply Healthy")

# --- DETAILED DATA INSPECTOR ---
print(f"\n{'='*80}")
print(f"      DETAILED ECU DATA (Supply)")
print(f"{'='*80}")
# Adjusted formatting for larger numbers
print(f"{'ECU':<6} | {'TIER':<8} | {'ASIL':<5} | {'CPU(DMIPS)':<11} | {'GPU(TOPS)':<10} | {'RAM(MB)':<9} | {'ROM(MB)':<9} | {'COST':<5}")
print("-" * 80)
for j in ECUS:
    print(f"{j:<6} | {ECU_TIERS[j]:<8} | {ECU_ASIL_CAPABILITY[j]:<5} | {MAX_CPU[j]:<11.0f} | {MAX_GPU[j]:<10.0f} | {MAX_RAM[j]:<9.0f} | {MAX_ROM[j]:<9.0f} | {ECU_BASE_COST[j]:<5.0f}")

print(f"\n{'='*80}")
print(f"      DETAILED COMPONENT DATA (Demand)")
print(f"{'='*80}")
print(f"{'COMP':<6} | {'TYPE':<12} | {'ASIL':<5} | {'CPU':<7} | {'GPU':<5} | {'RAM':<6} | {'ROM':<6} | {'REQ HW'}")
print("-" * 80)
# Showing first 15 components to keep log readable, remove slice [:15] to see all
for i in COMPONENTS: 
    hw = ",".join(REQ_HW[i]) if REQ_HW[i] else "-"
    # Format floats to 1 decimal place
    print(f"{i:<6} | {COMP_TYPES[i]:<12} | {REQ_ASIL[i]:<5} | {REQ_CPU[i]:<7.0f} | {REQ_GPU[i]:<5.1f} | {REQ_RAM[i]:<6.1f} | {REQ_ROM[i]:<6.1f} | {hw}")

print("\nData generation complete.")


      SCENARIO SUMMARY (Seed: 42)
CPU Supply (DMIPS): 838,000 | Demand: 350,190
GPU Supply (TOPS):  1,500 | Demand: 1,028
GPU Supply Healthy
Safety Nodes: 4 ECUs | Critical Tasks: 11
Safety Tier Supply Healthy

      DETAILED ECU DATA (Supply)
ECU    | TIER     | ASIL  | CPU(DMIPS)  | GPU(TOPS)  | RAM(MB)   | ROM(MB)   | COST 
--------------------------------------------------------------------------------
E_1    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_2    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_3    | HPC      | B     | 250000      | 500        | 65536     | 256000    | 200  
E_4    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 30   
E_5    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_6    | SAFETY   | D     | 16000       | 0          | 24        | 64        | 80   
E_7    | GATEWAY  | B     | 8000        | 0          | 8         | 16        | 

1. **Decision Variables**: 
    We define the primary binary assigment variable for Software Component (SC) and a secondary binary variable for ECU utilization.

    $x_{ij}$: Binary variable, $x_{ij}$ = 1 if **SC** $i$ is assigned to **ECU** $j$, and 0 otherwise.

    $y_j$: Binary variable, $y_j$ = 1 if **ECU** $j$ is utilized (at least one component asssigned to it) otherwise is 0. 

In [59]:
# --- GEKKO NON-LINEAR MODEL ---

print("Initializing Non-Linear Solver (GEKKO)...")
m = GEKKO(remote=False) 
m.options.SOLVER = 1    # APOPT Solver ( NONLINEAR Mixed-Integer Solver)


# Convert String IDs to Indices for GEKKO
C_ID = {name: i for i, name in enumerate(COMPONENTS)}
E_ID = {name: j for j, name in enumerate(ECUS)}

num_c = len(COMPONENTS)
num_e = len(ECUS)

# --- 2. Decision Variables ---
print("Creating Variables...")

# x[i,j]: Component i -> ECU j (Binary: 0 or 1)
# lb=0, ub=1, integer=True it means binary variable
x = m.Array(m.Var, (num_c, num_e), lb=0, ub=1, integer=True)

# y[j]: ECU j is Used? (Binary)
y = m.Array(m.Var, num_e, lb=0, ub=1, integer=True)

Initializing Non-Linear Solver (GEKKO)...
Creating Variables...


In [ ]:
# --- GEKKO NON-LINEAR MODEL (FINAL CORRECTED & SCALED) ---
from gekko import GEKKO

print("Initializing Scaled Non-Linear Solver...")
m = GEKKO(remote=False) 

# --- Değişkenler ---
# Integer=True ile baştan tanımlıyoruz (APOPT için)
x = m.Array(m.Var, (num_c, num_e), lb=0, ub=1, integer=True)
y = m.Array(m.Var, num_e, lb=0, ub=1, integer=True)

# --- Objective 1: Communication ($Z_1$) ---
z1_terms = []
for (c1_name, c2_name), vol in COMM_VOLUME.items():
    i, k = C_ID[c1_name], C_ID[c2_name]
    same_ecu = m.sum([x[i,j] * x[k,j] for j in range(num_e)])
    z1_terms.append(vol * (1 - same_ecu))
Z1 = m.sum(z1_terms)

# --- Objective 2: Load Balancing ($Z_2$ - SCALED) ---
# ÖLÇEKLEME (SCALING): Sayıları küçültüyoruz.
SCALE_FACTOR = 1000.0 
avg_cpu_target = sum(REQ_CPU.values()) / num_e / SCALE_FACTOR

z2_terms = []
for j in range(num_e):
    # Yükü scale ediyoruz
    load = m.sum([x[i,j] * (REQ_CPU[COMPONENTS[i]] / SCALE_FACTOR) for i in range(num_c)])
    # (Load - Target)^2 
    z2_terms.append((load - avg_cpu_target)**2) 
Z2 = m.sum(z2_terms)

# --- Objective 3: Cost ($Z_3$) ---
Z3 = m.sum([ECU_BASE_COST[ECUS[j]] * y[j] for j in range(num_e)])

# --- Kısıtlar ---
for i in range(num_c):
    m.Equation(m.sum([x[i,j] for j in range(num_e)]) == 1)

for j in range(num_e):
    # Kapasite kısıtları
    m.Equation(m.sum([x[i,j] * REQ_CPU[COMPONENTS[i]] for i in range(num_c)]) <= MAX_CPU[ECUS[j]])
    m.Equation(m.sum([x[i,j] * REQ_RAM[COMPONENTS[i]] for i in range(num_c)]) <= MAX_RAM[ECUS[j]])
    m.Equation(m.sum([x[i,j] * REQ_ROM[COMPONENTS[i]] for i in range(num_c)]) <= MAX_ROM[ECUS[j]])
    m.Equation(m.sum([x[i,j] * REQ_GPU[COMPONENTS[i]] for i in range(num_c)]) <= MAX_GPU[ECUS[j]])
    
    # Link Constraint
    m.Equation(m.sum([x[i,j] for i in range(num_c)]) <= num_c * y[j])

# Safety Isolation
for c1, c2 in CRITICAL_PAIRS_TO_SEPARATE:
    idx1, idx2 = C_ID[c1], C_ID[c2]
    for j in range(num_e):
        m.Equation(x[idx1,j] + x[idx2,j] <= 1)

# --- SOLVE ---
m.Minimize(LAMBDA_1 * Z1 + LAMBDA_2 * Z2 + LAMBDA_3 * Z3)

# 1. Aşama: Relaxed (Isınma Turu - IPOPT)
print("\n--- Phase 1: Relaxed Pre-Solve (IPOPT) ---")
m.options.SOLVER = 3 
try:
    m.solve(disp=False)
    print(f">> Pre-solve OK. Obj: {m.options.OBJFCNVAL:.2f}")
except:
    print(">> Pre-solve failed. Continuing anyway...")

# 2. Aşama: Integer (Gerçek Maç - APOPT)
print("\n--- Phase 2: Integer Enforcement (APOPT) ---")
m.options.SOLVER = 1 
# m.options.SOLVESTATUS = 0  <-- BU SATIR HATALIYDI, SİLDİK.

try:
    m.solve(disp=True)
except Exception as e:
    print("Solver finished with errors/warning.")

# --- SONUÇ KONTROLÜ ---
# m.solve() çalıştıktan sonra status otomatik güncellenir.
if m.options.SOLVESTATUS == 1:
    print(f"\n✅ REAL SUCCESS! Final Scaled Objective: {m.options.OBJFCNVAL:.2f}")
    
    print(f"\n{'='*60}")
    print(f"      FINAL ASSIGNMENTS")
    print(f"{'='*60}")
    
    for i in range(num_c):
        assigned = False
        for j in range(num_e):
            # Güvenli okuma
            raw = x[i,j].value
            # GEKKO bazen tek elemanlı liste, bazen direkt sayı döndürür
            val = raw[0] if isinstance(raw, (list, np.ndarray)) else raw
            
            if val > 0.9: 
                print(f"[{COMPONENTS[i]}] ({COMP_TYPES[COMPONENTS[i]]}) --> {ECUS[j]}")
                assigned = True
        if not assigned:
            print(f"❌ {COMPONENTS[i]} NOT ASSIGNED")

else:
    print(f"\n❌ FAILED. Status Code: {m.options.SOLVESTATUS}")
    print("Non-Linear Solver (APOPT) tam sayı çözümü bulamadı.")

Initializing Scaled Non-Linear Solver...

--- Phase 1: Relaxed Pre-Solve (IPOPT) ---
>> Pre-solve OK. Obj: 137221.19

--- Phase 2: Integer Enforcement (APOPT) ---


## Constraints (Feasibility/Design Criteria): 
These are the strict rules that must be satisfied for any valid solution.


### C1: Unique Assignment: 
Each component _i_ must be assigned to exactly one **ECU** 

$$\sum_{j=1}^{J} x_{ij} = 1 \quad \forall i \in \{1, \dots, I\}$$

In [60]:
for i in range(num_c):
    m.Equation(m.sum([x[i,j] for j in range(num_e)]) == 1)

### C2: Resource Capacity: 
Total required **_CPU/RAM/ROM_** resources by assigned components must not exceed the ECU's capacity.
$$\sum_{i=1}^{I} (\text{ReqResource}_{i, r} \cdot x_{ij}) \le \text{MaxCapacity}_{j, r} \quad \forall j \in \{1, \dots, J\}, \forall r$$

In [61]:
for j in range(num_e):
    e_name = ECUS[j] # E_1, E_2...
    
    # CPU 
    m.Equation(m.sum([x[i,j] * REQ_CPU[COMPONENTS[i]] for i in range(num_c)]) <= MAX_CPU[e_name])
    
    # RAM
    m.Equation(m.sum([x[i,j] * REQ_RAM[COMPONENTS[i]] for i in range(num_c)]) <= MAX_RAM[e_name])
    
    # ROM
    m.Equation(m.sum([x[i,j] * REQ_ROM[COMPONENTS[i]] for i in range(num_c)]) <= MAX_ROM[e_name])
    
    # GPU 
    m.Equation(m.sum([x[i,j] * REQ_GPU[COMPONENTS[i]] for i in range(num_c)]) <= MAX_GPU[e_name])

### C3: Safety Isolation (SPoF):

Critical redundant components ($i, k$) cannot be placed on the same ECU to prevent a Single Point of Failure.
$$x_{ij} + x_{kj} \le 1 \quad \forall j \in \{1, \dots, J\} \text{ if } (i, k) \text{ is a critical redundant pair}$$

In [62]:
for c1_name, c2_name in CRITICAL_PAIRS_TO_SEPARATE:
    idx1 = C_ID[c1_name]
    idx2 = C_ID[c2_name]
    for j in range(num_e):
        m.Equation(x[idx1,j] + x[idx2,j] <= 1)

### C4: Hardware Compatibility:
ConstraintComponent $i$ can only be assigned to ECU $j$ if the specific hardware or peripheral requirements (${Req_{HW}}^i$) of $i$ are available ($\text{AvailHW}_j$) on $j$.$$x_{ij} = 0 \quad \text{if } \text{ReqHW}_i \not\subseteq \text{AvailHW}_j$$

### C5: ASIL Safety Constraint: 
Logic: A Component cannot be assigned to an ECU with a lower ASIL Capability.
If Required_ASIL > Available_ASIL, then x[i][j] MUST be 0.

In [ ]:
for i in range(num_c):
    c_name = COMPONENTS[i]
    req_asil_val = ASIL_MAP[REQ_ASIL[c_name]]
    req_hw_set = set(REQ_HW.get(c_name, []))
    
    for j in range(num_e):
        e_name = ECUS[j]
        cap_asil_val = ASIL_MAP[ECU_ASIL_CAPABILITY[e_name]]
        cap_hw_set = set(ECU_HW_CAPABILITIES.get(e_name, []))
        
        # ASIL InCompatibility
        if req_asil_val > cap_asil_val:
            m.Equation(x[i,j] == 0)
            
        # HW Compatibility
        if not req_hw_set.issubset(cap_hw_set):
            m.Equation(x[i,j] == 0)

### C6: ECU Utilization Link:
The binary variable $y_j$ must be correctly linked to $x_{ij}$ to track active ECUs for the cost objective (O3). $M$ is a sufficiently large number (e.g., the total number of components $I$).
$$\sum_{i=1}^{I} x_{ij} \le M \cdot y_{j} \quad \forall j \in \{1, \dots, J\}$$

In [64]:
for j in range(num_e):
    m.Equation(m.sum([x[i,j] for i in range(num_c)]) <= num_c * y[j])

## Objective Functions
The problem is structured as a Multi-Objective Optimization. The final goal is to minimize a weighted sum of the following conflicting objectives:

### O1: Minimize Inter-ECU Communication Cost ($Z_1$): 
Minimize the total communication volume and associated latency penalties incurred when components $i$ and $k$ that communicate heavily are assigned to different ECUs ($j \neq l$).

$$\min Z_1 = \sum_{i=1}^{I} \sum_{k=1}^{I} \sum_{j=1}^{J} \sum_{l=1}^{J} (\text{CommVol}_{i,k} \cdot x_{ij} \cdot x_{kl} \cdot \delta_{j \neq l})$$

(Note: The term $x_{ij} \cdot x_{kl}$ is non-linear and requires linearization techniques or a non-linear solver.)

In [65]:
z1_terms = []

for (c1_name, c2_name), vol in COMM_VOLUME.items():
    i = C_ID[c1_name]
    k = C_ID[c2_name]
    
    # Same ECU? sum(x[i,j] * x[k,j])
    same_ecu = m.sum([x[i,j] * x[k,j] for j in range(num_e)])
    
    # if same_ecu = 1 cost = 0 
    # if same_ecu = 0 cost = vol
    cost_term = vol * (1 - same_ecu)
    z1_terms.append(cost_term)

Z1 = m.sum(z1_terms)

### O2: Minimize Resource Imbalance (Load Balancing) ($Z_2$):

Minimize the variance of resource utilization across all active ECUs to ensure balanced workload distribution, enhancing system responsiveness and stability.

$$\min Z_2 = \sum_{j=1}^{J} \left( \text{Utilization}_j - \text{AvgUtilization} \right)^2$$

(Note: This is a non-linear (quadratic) objective, often approximated or converted to a Minimax linear problem in ILP, or handled via meta-heuristics.) 

In [66]:

avg_cpu_target = sum(REQ_CPU.values()) / num_e
print(f"Average CPU Load Target per ECU: {avg_cpu_target:,.2f}")
z2_terms = []
avg_cpu_target = sum(REQ_CPU.values()) / num_e

for j in range(num_e):
    # Calculate load for ECU j
    load = m.sum([x[i,j] * REQ_CPU[COMPONENTS[i]] for i in range(num_c)])
    
    # Use m.abs3() which creates a linear internal structure for Absolute Value
    # Minimize |Load - Target| instead of (Load - Target)^2
    dev = m.Var(lb=0)
    m.Equation(dev == m.abs3(load - avg_cpu_target))
    z2_terms.append(dev)

Z2 = m.sum(z2_terms)

Average CPU Load Target per ECU: 35,019.01


### O3: Minimize Total System Cost and Complexity Index ($Z_3$)

Minimize the total hardware acquisition cost, factoring in penalties for excessive wiring/complexity and residual safety risk associated with the chosen ECU count.
$$\min Z_3 = \sum_{j=1}^{J} (\text{Cost}_j \cdot y_j) $$

In [67]:
# The total cost includes the base cost of the ECU plus the complexity/risk penalty 
# (represented here as a fixed amount added to the base cost, multiplied by the usage variable y_j)
Z3 = m.sum([ECU_BASE_COST[ECUS[j]] * y[j] for j in range(num_e)])

***Overall Optimization Goal (Weighted Sum Method)***
The combined objective is to minimize the weighted sum $Z$:$$\min Z = \lambda_1 \cdot Z_1 + \lambda_2 \cdot Z_2 + \lambda_3 \cdot Z_3$$($\lambda_1, \lambda_2, \lambda_3$ are pre-defined positive weighting factors determined by architectural priorities.)

### Solution

In [ ]:
# Weights 
LAMBDA_1 = 1.0 #Communication Cost Weight
LAMBDA_2 = 5.0 #Load Balancing Weight
LAMBDA_3 = 200.0 #Complexity Penalty Weight

# Combine the objectives using the defined weights (LAMBDA_1 and LAMBDA_3)
m.Minimize(LAMBDA_1 * Z1 + LAMBDA_2 * Z2 + LAMBDA_3 * Z3)

try:
    m.solve(disp=True)
    print("\nSuccessful Solution Found!")
except Exception as e:
    print("Solver stopped (Max time or Optimal found). Checking results...")


 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  247
   Constants    :  0
   Variables    :  4361
   Intermediates:  0
   Connections  :  4651
   Equations    :  3935
   Residuals    :  3935
 
 Number of state variables:    4361
 Number of total equations: -  4181
 Number of slack variables: -  80
 ---------------------------------------
 Degrees of freedom       :    100
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I: -1 Tm:     27.78 NLPi:  227 Dpth:    0 Lvs:    0 Obj:  0.00E+00 Gap:       NaN
 Maximum iterations
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :  27.802899999999998 sec
 Objective   

In [69]:
# --- ROBUST GEKKO RESULTS ANALYSIS ---

# 1. Solver Durumunu Kontrol Et
# SOLVESTATUS: 1=Başarılı, 0=Başarısız
if m.options.SOLVESTATUS == 1:
    print(f"\n✅ SUCCESS! Solution Found.")
    print(f"Final Objective Value: {m.options.OBJFCNVAL}")

    assignment_map = {}
    unassigned_comps = []

    print("\n--- Component Assignments ---")
    for i in range(num_c):
        assigned = False
        for j in range(num_e):
            # Değeri güvenli okuma (Liste veya Sayı)
            raw_val = x[i,j].value
            if isinstance(raw_val, (list, np.ndarray)):
                val = raw_val[0]
            else:
                val = raw_val
                
            if val > 0.9:
                c_name = COMPONENTS[i]
                e_name = ECUS[j]
                assignment_map[c_name] = e_name
                assigned = True
                break
        
        if not assigned:
            unassigned_comps.append(COMPONENTS[i])

    if unassigned_comps:
        print(f"⚠️ WARNING: {len(unassigned_comps)} components could not be assigned!")
        
    # Detaylı ECU Raporu
    print(f"\n{'='*60}")
    print(f"      ECU UTILIZATION REPORT")
    print(f"{'='*60}")
    print(f"{'ECU':<6} | {'TIER':<8} | {'CPU %':<8} | {'RAM %':<8} | {'GPU %':<8} | {'ROM %':<8}")
    print("-" * 60)

    for j in range(num_e):
        e_name = ECUS[j]
        
        # Y değerini güvenli okuma
        raw_y = y[j].value
        if isinstance(raw_y, (list, np.ndarray)):
            y_val = raw_y[0]
        else:
            y_val = raw_y
        
        if y_val > 0.9:
            my_comps = [c for c, e in assignment_map.items() if e == e_name]
            if not my_comps: continue
                
            cpu_used = sum(REQ_CPU[c] for c in my_comps)
            ram_used = sum(REQ_RAM[c] for c in my_comps)
            gpu_used = sum(REQ_GPU[c] for c in my_comps)
            rom_used = sum(REQ_ROM[c] for c in my_comps)
            
            cpu_p = (cpu_used / MAX_CPU[e_name]) * 100
            ram_p = (ram_used / MAX_RAM[e_name]) * 100
            rom_p = (rom_used / MAX_ROM[e_name]) * 100
            
            gpu_p = 0.0
            if MAX_GPU[e_name] > 0:
                gpu_p = (gpu_used / MAX_GPU[e_name]) * 100
                
            tier = ECU_TIERS[e_name]
            print(f"{e_name:<6} | {tier:<8} | {cpu_p:<6.1f} % | {ram_p:<6.1f} % | {gpu_p:<6.1f} % | {rom_p:<6.1f} %")
        else:
            print(f"{e_name:<6} | {'(OFF)':<8} | -        | -        | -        | -")

else:
    # --- SOLVER BAŞARISIZ OLURSA BURASI ÇALIŞIR ---
    print(f"\n{'='*50}")
    print(f"❌ SOLVER FAILED TO FIND A SOLUTION")
    print(f"{'='*50}")
    print("Possible reasons for Infeasibility:")
    print("1. GPU Shortage: Check 'Scenario Summary' above. (Demand > Supply?)")
    print("2. Safety Constraints: Are there enough ASIL D ECUs for critical tasks?")
    print("3. Time Limit: Problem might be too complex for the default time limit.")
    print("\nSuggested Fix:")
    print("-> Go back to 'Data Generation' step.")
    print("-> Increase COUNT_HPC or reduce WEIGHT_PERCEPTION.")
    print("-> Re-run the notebook.")

        


✅ SUCCESS! Solution Found.
Final Objective Value: 0.0

--- Component Assignments ---


TypeError: 'int' object is not subscriptable